*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
# try:
  # %tensorflow_version only exists in Colab.
  # !pip install tf-nightly
# except Exception:
#   pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
# !pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.5.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-06-30 00:38:20--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.006s  

2021-06-30 00:38:21 (61.3 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2021-06-30 00:38:21--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[========

## Import the dataset into train_df and test_df

In [ ]:
train_df = pd.read_csv(train_file_path,sep='\t',  names=['type', 'text'] )
test_df = pd.read_csv(test_file_path,sep='\t',  names=['type', 'text'] )

## Preprocessing
The first step in preprocessing is to encode the text into integer sequences si that they can be fed into the mearal network. Fortunately keras has a class called **Tokenizer** that do this for us.

In [ ]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_df['text'])

In [ ]:
MAXLEN = max([len(i) for i in tokenizer.texts_to_sequences(train_df['text'])])
VOCAB_SIZE = len(tokenizer.index_word) + 1

In [ ]:
# Encode the train and test datasets
enc_train = tokenizer.texts_to_sequences(train_df['text'])
enc_test = tokenizer.texts_to_sequences(test_df['text'])

Now we pad the length of the encoded texts so that all of them will be the same length. All the observations need to be the same length to work in a neural network. We use the length of the observation with the maximum length.



In [ ]:
enc_train = keras.preprocessing.sequence.pad_sequences(enc_train, MAXLEN)
enc_test = keras.preprocessing.sequence.pad_sequences(enc_test, MAXLEN)

## Model Training
For this project we are using a neural network with three layers.

1.   The first layer is an embedding layer, with an input dimension of 7693. This is the number of different words in our dataset. It has an output dimension of 32
2.   The second layer is a Long Short-Term Memory (LSTM) layer. This layer allows the network to learn the order dependence in our text sequence. This is allows the network to remember the word order in our text sequences.
3.   The final layer is a dense layer with one output. It use a sigmoid as the activation function this squishes the result to a value between 0 and 1 and allow us to get the probality of a word being 'ham'(0) or 'spam'(1)

In [ ]:
model = keras.Sequential([
    tf.keras.layers.Embedding( VOCAB_SIZE, 32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, None, 32)          246176    
_________________________________________________________________
lstm_8 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 33        
Total params: 254,529
Trainable params: 254,529
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

In [ ]:
type_numpy = train_df.type.replace({'ham':0, 'spam':1}).to_numpy()

In [ ]:
history = model.fit( enc_train, type_numpy, epochs=10, validation_split=0.2)

Epoch 1/10
105/105 [==============================] - 10s 82ms/step - loss: 0.2548 - acc: 0.9049 - val_loss: 0.1124 - val_acc: 0.9725
Epoch 2/10
105/105 [==============================] - 8s 77ms/step - loss: 0.0740 - acc: 0.9838 - val_loss: 0.0539 - val_acc: 0.9880
Epoch 3/10
105/105 [==============================] - 8s 77ms/step - loss: 0.0391 - acc: 0.9901 - val_loss: 0.0449 - val_acc: 0.9880
Epoch 4/10
105/105 [==============================] - 8s 76ms/step - loss: 0.0261 - acc: 0.9928 - val_loss: 0.0745 - val_acc: 0.9761
Epoch 5/10
105/105 [==============================] - 8s 76ms/step - loss: 0.0192 - acc: 0.9940 - val_loss: 0.0401 - val_acc: 0.9904
Epoch 6/10
105/105 [==============================] - 8s 76ms/step - loss: 0.0125 - acc: 0.9961 - val_loss: 0.0553 - val_acc: 0.9892
Epoch 7/10
105/105 [==============================] - 8s 77ms/step - loss: 0.0102 - acc: 0.9973 - val_loss: 0.0460 - val_acc: 0.9916
Epoch 8/10
105/105 [==============================] - 8s 77ms/step -

## Model Evaluation

In [ ]:
model.evaluate(enc_test, test_df.type.replace({'ham':0, 'spam':1}).to_numpy())

44/44 [==============================] - 1s 16ms/step - loss: 0.0565 - acc: 0.9878


[0.05653685703873634, 0.9877873659133911]

In [ ]:
def message_type(score):
  if score > 0.5:
    return 'spam'
  return 'ham'

# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  enc_text = tokenizer.texts_to_sequences([pred_text])
  enc_text = keras.preprocessing.sequence.pad_sequences(enc_text, MAXLEN) 
  score = round(model.predict(enc_text)[0][0], 2)
  prediction = [score, message_type(score)]

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

(0.0, 'ham')


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
